# Hopsworks Feature Store - Part 03 - Model training, Lineage.

In this notebook, we'll train a model on the dataset we created in the previous tutorial. We will train our model using standard Python and Scikit-learn, although it could just as well be trained with other machine learning frameworks such as PySpark, TensorFlow, and PyTorch.

![tutorial-flow](images/model_training_other.png)

In [ ]:
import hsfs

conn = hsfs.connection()
fs = conn.get_feature_store()

### Load Training Data

First, we'll need to fetch the training dataset that we created in the previous notebook.

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder

# Load data.
feature_view = fs.get_feature_view("transactions_view", 1)
_, td_df = feature_view.get_training_dataset_splits({'train': 80, 'validation': 20}, start_time=None, end_time=None, version = 1)

td = fs.get_training_dataset("transactions_view_1",1)
X_train = td_df["train"]
X_val = td_df['validation']


We will train a model to predict `fraud_label` given the rest of the features.

In [ ]:
# Separate target feature from input features.
target = target = feature_view.label[0]  
y_train = X_train.pop(target)
y_val = X_val.pop(target)

Let's check the distribution of our target label.

In [ ]:
y_train.value_counts(normalize=True)

Notice that the distribution is extremely skewed, which is natural considering that fraudulent transactions make up a tiny part of all transactions. Thus we should somehow address the class imbalance. There are many approaches for this, such as weighting the loss function, over- or undersampling, creating synthetic data, or modifying the decision threshold. In this example, we'll use the simplest method which is to just supply a class weight parameter to our learning algorithm. The class weight will affect how much importance is attached to each class, which in our case means that higher importance will be placed on positive (fraudulent) samples.

### Train Model

Next we'll train a model. Here, we set the class weight of the positive class to be twice as big as the negative class.

In [ ]:
# Train model.
clf = LogisticRegression(class_weight={0: 1.0 - pos_class_weight, 1: pos_class_weight}, solver='liblinear')
clf.fit(X_train, y_train)

Let's see how well it performs on our validation data.

In [ ]:
from sklearn.metrics import classification_report

preds = clf.predict(X_val)

print(classification_report(y_val, preds))


### Wrapping up

We have now performed a simple training with training data that we have created in the feature store. This will also allow us to navigate in the lineage graph as seen in the following GIF. 

[//]: <> (insert GIF here)

This concludes the fisrt module and introduction to the core aspect of the feauture store. In the second module we will introduce streaming and external feature groups for a similar fraud use case.